In [8]:

import numpy as np
from lmfit import minimize, Parameters, fit_report
import matplotlib.pyplot as plt
import pandas as pd
from typing import List

from transformations import euler_matrix, euler_from_matrix, scale_from_matrix

def convert_to_homogeneous(x: np.ndarray) -> np.ndarray:
    ''' Convert 3d points in euclidean coordinates (nx3 numpy array) homogenous coordinates (nx4 numpy array)
    '''
    if x.shape[1] != 3: 
        raise ValueError('Wrong dimension of the input array, please provide nx3 numpy array')
    n = x.shape[0]
    x = np.concatenate((x, np.ones((n, 1))), 1, )
     
    return x

def residuals(params: Parameters,
              x0: np.ndarray, # Observed values (original reference system) 
              x1: np.ndarray, # Estimated values (final reference system)
              uncertainty: np.ndarray = None # A-priori observation uncertainty
            ) -> np.ndarray:
    # @TODO: Check for correct order of observed and estimated!
    ''' 3D rototranslation with scale factor
        X1_ = T_ + m * R * X0_
    '''    
        
    # Get parameters  
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    m = parvals['m']
    
    # Convert points to homogeneos coordinates and traspose np array to obtain a 4xn matrix
    x0 = convert_to_homogeneous(x0).T

    # Build 4x4 transformation matrix (T) in homogeneous coordinates
    T = np.identity(4)
    R = euler_matrix(rx, ry, rz)
    T[0:3, 0:3] = (m * np.identity(3)) @ R[:3,:3]
    T[0:3, 3:4] = np.array([tx, ty, tz]).reshape(3, 1)
    
    # Apply transformation to x0 points
    x1_ = T @ x0
    x1_ = x1_[:3,:].T 

    # Compute residuals as differences between observed and estimated values, scaled by the a-priori observation uncertainties
    res = (x1 - x1_) # / uncertainty

    # if uncertainty is None:
    #     uncertainty = np.ones_like(x0, dtype='float64')
        
    
    # 
    # num_pts = x0.shape[0]
    
    # # # Build rotation matrix (R), translation vector (t) and 4x4 transformation matrix (T) in homogeneous coordinates
    # R = euler_matrix(rx, ry, rz)
    
    # #  
    # # i = 0
    # res = []
    # for i in range(num_pts):
    #     x_ = tx + m * np.matmul(R[0, :3].reshape(1, 3), x0[i,:].reshape(3, 1))
    #     y_ = ty + m * np.matmul(R[1, :3].reshape(1, 3), x0[i,:].reshape(3, 1))
    #     z_ = tz + m * np.matmul(R[2, :3].reshape(1, 3), x0[i,:].reshape(3, 1))
        
    #     rx = x1[i, 0] - x_[0][0]
    #     ry = x1[i, 1] - y_[0][0]
    #     rz = x1[i, 2] - z_[0][0]
    #     res_i = np.array([rx, ry, rz])     
    #     res.append(res_i)
   
    return np.array(res).flatten()

# minner = Minimizer(residuals, params)
# result = minner.minimize(method='leastsq')


def read_data_to_df(
    file_path: str,
    delimiter: str = ',',
    header: int = 0,
    col_names: List[str] = None,
    index_col=None,
) -> pd.DataFrame:
    '''
    '''
    df = pd.read_csv(file_path,
                     sep=delimiter,
                     header=header,
                     names=col_names,
                     index_col=index_col)
    return df


pt_loc = read_data_to_df('lingua_loc.txt',
                         header=None,
                         col_names=['label', 'x', 'y', 'z'],
                         index_col=0
                         )
# pt_loc.to_numpy()
pt_utm = read_data_to_df('lingua_utm.txt',
                         header=None,
                         col_names=['label', 'x', 'y', 'z'],
                         index_col=0
                         )
# pt_utm.to_numpy()

t_ini = np.array([16600., 90900., 1700.], dtype='float64')
rot_ini = np.array([0., 0., 0.], 'float64')
m_ini = float(1.0)

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot_ini[0], vary=False)
params.add('ry', value=rot_ini[1], vary=False)
params.add('rz', value=rot_ini[2], vary=True)
params.add('tx', value=t_ini[0], vary=True)
params.add('ty', value=t_ini[1], vary=True)
params.add('tz', value=t_ini[2], vary=True)
params.add('m',  value=m_ini, vary=True)

# Run Optimization!
result = minimize(residuals, params, args=(
    pt_loc.to_numpy(), pt_utm.to_numpy())
)

# Print result
print(fit_report(result))
print(f'Residuals:\n {result.residual.reshape(-1,3)}')
# print(f'Covariance matrix: \n {result.covar}')

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 31
    # data points      = 12
    # variables        = 5
    chi-square         = 2.5946e-04
    reduced chi-square = 3.7066e-05
    Akaike info crit   = -118.901773
    Bayesian info crit = -116.477239
[[Variables]]
    rx:  0 (fixed)
    ry:  0 (fixed)
    rz:  0.78559307 +/- 8.1183e-05 (0.01%) (init = 0)
    tx:  16614.8211 +/- 0.01502196 (0.00%) (init = 16600)
    ty:  90932.7197 +/- 0.01411835 (0.00%) (init = 90900)
    tz:  1767.50990 +/- 0.00687770 (0.00%) (init = 1700)
    m:   0.99947568 +/- 7.6039e-05 (0.01%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(rz, tx) = 0.979
    C(ty, m)  = -0.976
    C(tz, m)  = -0.897
    C(ty, tz) = 0.875
Residuals:
 [[-0.00444834 -0.00156923  0.00479643]
 [-0.00035528 -0.00110734 -0.00714325]
 [ 0.00205395  0.00321977  0.00943872]
 [ 0.00274967 -0.00054321 -0.00709189]]


/home/francesco/anaconda3/envs/rotra3d/lib/python3.8/site-packages/lmfit/printfuncs.py:175: RuntimeWarning: invalid value encountered in double_scalars
  spercent = f'({abs(par.stderr/par.value):.2%})'
